# Optimisation: Compilation

Compilation is the act of a program, known as a compiler, examining source code and creating a new piece of executable code which may be executed later. Compilation offers a few useful advantages. A compiler can check the syntax of all the code of a program before it is run, which can aid finding problems in the code. Of more immediate use to us, however, is the fact that compilers can interpret our code and write a more efficient piece of code in a lower level language which is guaranteed to produce the same result. This is the property of compilation that we can use to speed up code written in Python.

Some languages are, by default, compiled languages, such as Fortran and C. Code written in these languages are usually compiled before they are run. In these cases, a separate exceutable file is normally created and this is what is run, without reference to the source code at all.

Python code can be compiled this way, but there are other, more convenient ways of using the idea of compilation to speed up Python code.

## Numba

[Numba](http://numba.pydata.org/) is an example of a Python tool which harnesses the power of compilation to speed up existing Python code with minimal changes to code required. Many of the features in Numba are accessed by adding various decorators to functions. Let's look at a simple example now.

The following example is taken from a series of short examples from the [Numba website](https://numba.pydata.org/). The first code calcualtes the value of $\pi$ using the same method we used in the Parallelism notebook. In the second code cell we import the ```jit``` decorator from the ```numba``` package and add the decorator ```@jit(nopython=True)``` to the function. This causes Numba to compile the function just before it's run and for Python to run the compiled function. Here, ```jit``` stands for "just in time" as the compilation is happening just before the function is run. The ```nonpython = true``` forces Numba to attempt to compile the function using faster, non-Python, code.

If you're running this Jupyter Notebook on your computer, as opposed to Colab, you may need to install the Numba package before you can run the code.

In [1]:
import random
import cProfile

def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

cProfile.run('print(monte_carlo_pi(10000000))')

3.1422624
         20000039 function calls in 11.381 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    9.147    9.147   11.379   11.379 <ipython-input-1-c431711fad5c>:4(monte_carlo_pi)
        1    0.000    0.000   11.380   11.380 <string>:1(<module>)
        3    0.000    0.000    0.002    0.001 iostream.py:195(schedule)
        2    0.000    0.000    0.000    0.000 iostream.py:307(_is_master_process)
        2    0.000    0.000    0.001    0.001 iostream.py:320(_schedule_flush)
        2    0.000    0.000    0.002    0.001 iostream.py:382(write)
        3    0.000    0.000    0.000    0.000 iostream.py:93(_event_pipe)
        3    0.002    0.001    0.002    0.001 socket.py:540(send)
        3    0.000    0.000    0.000    0.000 threading.py:1050(_wait_for_tstate_lock)
        3    0.000    0.000    0.000    0.000 threading.py:1092(is_alive)
        3    0.000    0.000    0.000    0.000 threading.py:507(is_set)


In [2]:
# Import the jit decorator from the numba package
from numba import jit
import random
import cProfile

# Apply the jit decorator to our function
@jit(nopython=True)
def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

cProfile.run('monte_carlo_pi(10000000)')

         481993 function calls (455514 primitive calls) in 1.068 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      976    0.001    0.000    0.003    0.000 <__array_function__ internals>:2(can_cast)
1924/1253    0.002    0.000    0.146    0.000 <frozen importlib._bootstrap>:1009(_handle_fromlist)
      147    0.000    0.000    0.001    0.000 <frozen importlib._bootstrap>:103(release)
      116    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:143(__init__)
      116    0.000    0.000    0.001    0.000 <frozen importlib._bootstrap>:147(__enter__)
      116    0.000    0.000    0.001    0.000 <frozen importlib._bootstrap>:151(__exit__)
      147    0.001    0.000    0.001    0.000 <frozen importlib._bootstrap>:157(_get_module_lock)
      115    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:176(cb)
       31    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:194(_lock_unlock_m

When we run the two pieces of code, we notice two things. Firstly, the version with the Numba decortator compiles much faster (when I ran it, about 0.3s compared to about 6s). The second is that the list of called functions is much longer in the compiled version. This is because cProfile has recorded the process of compilation as well as the running of the code. Most of this complexity is simply the detail of the implementation and can be ignored.

The important thing to notice is that Numba was able to speed up the execution of this code significantly with minimal changes to the source code required. This makes Numba a good choice to easily get some extra speed, by compiling heavily used functions in your code.